In [133]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import MetricFrame, selection_rate, count

train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
train_X = list(train_data['TFIDF'])
train_y = (train_data['Sentiment'] == 'positive') * 1

dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
dev_X = list(dev_data['TFIDF'])
dev_y  = (dev_data['Sentiment'] == 'positive') * 1



### Prediction results

In [134]:
lr = LogisticRegression(max_iter=500)
lr.fit(train_X, train_y)
labels_predict = lr.predict(dev_X)
print(classification_report(dev_y, labels_predict))
print(confusion_matrix(dev_y, labels_predict))

              precision    recall  f1-score   support

           0       0.71      0.68      0.69      2000
           1       0.69      0.72      0.70      2000

    accuracy                           0.70      4000
   macro avg       0.70      0.70      0.70      4000
weighted avg       0.70      0.70      0.70      4000

[[1358  642]
 [ 565 1435]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [135]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.69825
count          4000
dtype: object
            accuracy count
Demographic               
AAE           0.6645  2000
SAE            0.732  2000


#### 2. Selection rate + Demographic parity difference

In [136]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

0.51925
Demographic
AAE    0.5655
SAE     0.473
Name: selection_rate, dtype: object


In [137]:
from fairlearn.metrics import demographic_parity_difference

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=labels_predict, sensitive_features=dev_data['Demographic'])
print("Difference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 2)))

Difference between selection rate
Demographic parity difference: 0.09


#### 3 Error Rate Equality Difference

#### 3.1 Overall

In [138]:
confusion = confusion_matrix(dev_y, labels_predict)
print(confusion[0])
false_positive_rate = confusion[0][1]/sum(confusion[0])
print("False positive rate:", false_positive_rate)

print(confusion[1])
false_negative_rate = confusion[1][0]/sum(confusion[1])
print("False negative rate:", false_negative_rate)

[1358  642]
False positive rate: 0.321
[ 565 1435]
False negative rate: 0.2825


#### 3.2 AAE

In [139]:
# Filter and extract all AAE data
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
AAE_train_data = train_data[train_data['Demographic'] == 'AAE']
AAE_train_X = list(AAE_train_data['TFIDF'])
AAE_train_y = (AAE_train_data['Sentiment'] == 'positive') * 1

dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
AAE_dev_data = dev_data[dev_data['Demographic'] == 'AAE']
AAE_dev_X = list(AAE_dev_data['TFIDF'])
AAE_dev_y  = (AAE_dev_data['Sentiment'] == 'positive') * 1

In [140]:
AAE_labels_predict = lr.predict(AAE_dev_X)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(confusion_matrix(AAE_dev_y, AAE_labels_predict))

              precision    recall  f1-score   support

           0       0.69      0.60      0.64      1000
           1       0.65      0.73      0.69      1000

    accuracy                           0.66      2000
   macro avg       0.67      0.66      0.66      2000
weighted avg       0.67      0.66      0.66      2000

[[599 401]
 [270 730]]


In [141]:
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(AAE_confusion[0])
AAE_false_positive_rate = AAE_confusion[0][1]/sum(AAE_confusion[0])
print("False positive rate:", AAE_false_positive_rate)

print(AAE_confusion[1])
AAE_false_negative_rate = AAE_confusion[1][0]/sum(AAE_confusion[1])
print("False negative rate:", AAE_false_negative_rate)

[599 401]
False positive rate: 0.401
[270 730]
False negative rate: 0.27


#### 3.3 SAE

In [142]:
# Filter and extract all AAE data
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
SAE_train_data = train_data[train_data['Demographic'] == 'SAE']
SAE_train_X = list(SAE_train_data['TFIDF'])
SAE_train_y = (SAE_train_data['Sentiment'] == 'positive') * 1

dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
SAE_dev_data = dev_data[dev_data['Demographic'] == 'SAE']
SAE_dev_X = list(SAE_dev_data['TFIDF'])
SAE_dev_y  = (SAE_dev_data['Sentiment'] == 'positive') * 1

In [143]:
SAE_labels_predict = lr.predict(SAE_dev_X)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(confusion_matrix(SAE_dev_y, SAE_labels_predict))

              precision    recall  f1-score   support

           0       0.72      0.76      0.74      1000
           1       0.75      0.70      0.72      1000

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

[[759 241]
 [295 705]]


In [144]:
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(SAE_confusion[0])
SAE_false_positive_rate = SAE_confusion[0][1]/sum(SAE_confusion[0])
print("False positive rate:", SAE_false_positive_rate)

print(SAE_confusion[1])
SAE_false_negative_rate = SAE_confusion[1][0]/sum(SAE_confusion[1])
print("False negative rate:", SAE_false_negative_rate)

[759 241]
False positive rate: 0.241
[295 705]
False negative rate: 0.295
